In [9]:
import chromadb
chroma_client = chromadb.Client()

RuntimeError: Chroma is running in http-only client mode, and can only be run with 'chromadb.api.fastapi.FastAPI' or 'chromadb.api.async_fastapi.AsyncFastAPI' as the chroma_api_impl.             see https://docs.trychroma.com/guides#using-the-python-http-only-client for more information.

In [4]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.docstore.document import Document

loader = DirectoryLoader(path="../data/", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()

In [5]:
type(docs)

list

In [7]:
for doc in docs:
    print(type(doc))

<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_core.documents.base.Document'>
<class 'langchain_co

In [1]:
import json
import os

import chromadb
import nest_asyncio
from datasets import Dataset
import openai
from datasets import load_dataset
from dotenv import load_dotenv
from ragas.testset import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas import RunConfig
from ragas.metrics import (context_precision, context_recall)
from tqdm import tqdm

from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from langchain.docstore.document import Document as LangchainDocument
from ragas import evaluate

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
LLM_MODEL = "gpt35turbo"
EMBEDDING_MODEL = "ada0021_6"

storage_path = "../vectordb"

chroma_client = chromadb.PersistentClient(storage_path)
collection = "test"
chroma_vector_search_index_name = "vector_index"

#ds = load_dataset("m-ric/huggingface_doc", split="train")


metrics = [context_precision, context_recall]


llm_1 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_version="2024-02-01",
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model=LLM_MODEL,
    temperature=0
    )

llm_2 = AzureChatOpenAI(
    openai_api_type="azure",
    openai_api_version="2024-02-01",
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model=LLM_MODEL,
    temperature=0
)

azure_embeddings = AzureOpenAIEmbeddings(
    openai_api_type="azure",
    openai_api_version="2024-02-01",
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    model=EMBEDDING_MODEL,
    allowed_special={'<|endoftext|>'}
)

In [3]:
generator = TestsetGenerator.from_langchain(llm_1, llm_2, azure_embeddings)

In [4]:
distributions = {simple: 0.5, multi_context: 0.4, reasoning: 0.1}

In [6]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

loader = DirectoryLoader("../data/", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()

In [7]:
testdata = generator.generate_with_langchain_docs(docs, 10, distributions, run_config=RunConfig())

embedding nodes:   0%|          | 0/858 [00:00<?, ?it/s]

Filename and doc_id are the same for all nodes.


Generating:   0%|          | 0/10 [00:00<?, ?it/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
type(testdata)

ragas.testset.generator.TestDataset

In [20]:
testdata.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How can a function return a list of values in ...,[Page 41 of 63 8.2 Returning more than one val...,The return statement in Python can take a list...,simple,"[{'source': '..\data\basics.pdf', 'page': 40}]",True
1,How does the concept of kata contribute to cre...,[372 • The Phoenix ProjectThe most obvious ma...,The concept of kata contributes to creating a ...,simple,"[{'source': '..\data\phoenix.pdf', 'page': 372}]",True
2,What is the meaning of the conversion specifie...,[Page 19 of 63 a = 1.23 b = 1.2345e+002 Here ...,"The conversion specifier ""e"" or ""E"" in Python ...",simple,"[{'source': '..\data\basics.pdf', 'page': 18}]",True
3,What is the difference between tuples and list...,[Page 28 of 63 6.1.5 Tuples Tuples are lists t...,"Tuples are lists that cannot be changed, while...",simple,"[{'source': '..\data\basics.pdf', 'page': 27}]",True
4,What is the main character's name in the Clint...,[Chapter 24 • 239was like getting rid of a tho...,The main character's name in the Clint Eastwoo...,simple,"[{'source': '..\data\phoenix.pdf', 'page': 239}]",True
5,Why is understanding customer needs important ...,[284 • The Phoenix Projectkeep it clean. Noth...,Understanding customer needs is important for ...,multi_context,"[{'source': '..\data\phoenix.pdf', 'page': 284}]",True
6,How can custom reports help with the payroll d...,[28 • The Phoenix Projectpay period. I’ve bee...,Custom reports can help with the payroll data ...,multi_context,"[{'source': '..\data\phoenix.pdf', 'page': 28}]",True


In [18]:
test.question.to_list()

['How can a function return a list of values in Python?',
 'How does the concept of kata contribute to creating a culture of relentless improvement in an organization?',
 'What is the meaning of the conversion specifier "e" or "E" in Python?',
 'What is the difference between tuples and lists in Python?',
 "What is the main character's name in the Clint Eastwood movie Pale Rider?",
 'Why is understanding customer needs important for revenue, market share, and managing IT risks as business risks?',
 'How can custom reports help with the payroll data issue in Finance?']